<a href="https://colab.research.google.com/github/meghasharma1234/driver-distraction-det/blob/main/tryinf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout, Input, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import os
os.chdir('gdrive/My Drive/major pro/major_5000_crop')
from PIL import Image

In [ ]:
class Configuration:
    def __init__(self):
        self.epochs = 1
        self.batch_size = 16
        self.maxwidth =0
        self.maxheight=0
        self.minwidth = 35000
        self.minheight = 35000
        self.imgcount=0
        self.img_width_adjust = 227
        self.img_height_adjust= 227
        #Kaggle
        self.data_dir = "train"

In [ ]:
config = Configuration()

In [ ]:
def findPictureDims(path):
    for subdir, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".jpg"):
                config.imgcount+=1
                filename = os.path.join(subdir, file)
                image = Image.open(filename)
                width, height = image.size
                if width < config.minwidth:
                    config.minwidth = width
                if height < config.minheight:
                    config.minheight = height
                if width > config.maxwidth:
                    config.maxwidth = width
                if height > config.maxheight:
                    config.maxheight = height
    return

In [ ]:
def listDirectoryCounts(path):
    d = []
    for subdir, dirs, files in os.walk(path,topdown=False):
        filecount = len(files)
        dirname = subdir
        d.append((dirname,filecount))
    return d 

In [ ]:
def SplitCat(df):
    for index, row in df.iterrows():
        directory=row['Category'].split('/')
        if directory[3]!='':
            directory=directory[3]
            df.at[index,'Category']=directory
        else:
            df.drop(index, inplace=True)
    return

In [ ]:
def build_model():
    inputs = Input(shape=(config.img_width_adjust,config.img_height_adjust,3), name="input")
    
    #Convolution 1
    conv1 = Conv2D(96, kernel_size=(11,11), activation="relu",strides=(2,2), name="conv_1")(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2), name="pool_1")(conv1)

    #Convolution 2
    conv2 = Conv2D(256, kernel_size=(11,11),strides=(2,2), activation="relu", name="conv_2")(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2), name="pool_2")(conv2)
    
    #Convolution 3
    conv3 = Conv2D(384, kernel_size=(3,3), activation="relu", name="conv_3")(pool2)
   
    #Convolution 4
    conv4 = Conv2D(384, kernel_size=(3,3), activation="relu", name="conv_4")(conv3)
    
    
   #Convolution 5
    conv5 = Conv2D(256, kernel_size=(3,3), activation="relu", name="conv_5")(conv4)
    pool5 = MaxPooling2D(pool_size=(2, 2), name="pool_5")(conv5)
    
    #Fully Connected Layer
    flatten = Flatten()(pool5)
    fc1 = Dense(1000, activation="relu", name="fc_1")(flatten)
    
    #output
    output=Dense(6, activation="softmax", name ="softmax")(fc1)
    
    # finalize and compile
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
    return model


In [ ]:
def setup_data(train_data_dir, val_data_dir, img_width=config.img_width_adjust, img_height=config.img_height_adjust, batch_size=config.batch_size):
    
    train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split
    

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')
    
    validation_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation')
        #Note uses training dataflow generator
    return train_generator, validation_generator

In [ ]:
def fit_model(model, train_generator, val_generator, batch_size, epochs):
    model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size,
        verbose=1)
    return model

In [ ]:
def eval_model(model, val_generator, batch_size):
    scores = model.evaluate_generator(val_generator, steps=val_generator.samples // batch_size)
    print("Loss: " + str(scores[0]) + " Accuracy: " + str(scores[1]))

In [ ]:
train_generator, val_generator = setup_data(config.data_dir, config.data_dir, batch_size=config.batch_size)

Found 11932 images belonging to 6 classes.
Found 2980 images belonging to 6 classes.


In [ ]:
model = build_model()
print (model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 227, 227, 3)       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 109, 109, 96)      34944     
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 54, 54, 96)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 22, 22, 256)       2973952   
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 11, 11, 256)       0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 9, 9, 384)         885120    
_________________________________________________________________
conv

In [ ]:
model = fit_model(model, train_generator, val_generator,
                  batch_size=config.batch_size,
                  epochs=config.epochs)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
 20/745 [..............................] - ETA: 1:09:12 - loss: 3.3207 - acc: 0.1437

In [ ]:
model.save("tryinf.h5")

OSError: ignored

In [ ]:
!pip install -q h5py pyyaml

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

     |████████████████████████████████| 993kB 3.5MB/s 


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()                      
drive = GoogleDrive(gauth)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [ ]:
model.save('model.h5')
model_file = drive.CreateFile({'title' : 'model.h5'})                      
model_file.SetContentFile('model.h5')                      
model_file.Upload()

OSError: ignored

In [ ]:
model.save('model')

OSError: ignored

In [ ]:
eval_model(model, val_generator, batch_size=config.batch_size)

Loss: 3.2854261436925727 Accuracy: 0.5668016194331984


In [ ]:
model.save('try.h5')

NameError: ignored